[View in Colaboratory](https://colab.research.google.com/github/NaNkotsukan/deepLearningWithChainer/blob/master/DCGANwithChainer_ipynb_.ipynb)

## 環境設定

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80 chainer matplotlib
!nvidia-smi

# GANに依る画像生成
GAN(Generative adversarial networks)は複数のネットワークを競わせて学習し画像の生成などを行うモデルである．
提唱論文：https://arxiv.org/abs/1406.2661
今回使うモデルはDCGAN( https://arxiv.org/abs/1511.06434 )である．

In [0]:
from chainer import Chain, serializers, computational_graph
from chainer import Variable as V
import chainer.functions as F
import chainer.links as L
import chainer.computational_graph as C
from chainer.optimizers import Adam, SGD
import chainer

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display

import os
import numpy as np
import cupy as cp

%matplotlib inline

In [0]:
train, test = chainer.datasets.get_mnist(ndim=3)
trainX = train._datasets[0]
trainT = train._datasets[1]
testX = test._datasets[0]
testT = test._datasets[1]

print(trainX.shape)
print(testX.shape)

In [0]:
def show_img(img, title=None):
    fig = plt.figure(figsize=(15, 20))
    for i in range(len(img)):
        if img[i].ndim==3: x = img[i][0]
        else: x = img[i]
        a = fig.add_subplot(1, len(img), i+1)
        a.imshow(x)
        plt.xlim((0, 28))
        plt.ylim((28, 0))
        plt.tick_params(labelbottom=False, labelleft=False)
        if title != None:
            plt.title(title[i], fontsize=15)
        plt.grid()
    plt.show()

## DCGAN
DCGANではGeneratorとDiscriminatorの2つのモデルを使う．

In [0]:
class Generator(chainer.Chain):
    def __init__(self, n_hidden=100):
        super(Generator, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(in_size=n_hidden, out_size=512*9, initialW=w)
            self.deconv0 = L.Deconvolution2D(in_channels=512, out_channels=256, ksize=2, stride=2, pad=1)
            self.deconv1 = L.Deconvolution2D(256, 128, 2, 2, 1)
            self.deconv2 = L.Deconvolution2D(128, 64,  2, 2, 1)
            self.deconv3 = L.Deconvolution2D(64,  1,   3, 3, 1)
            self.bn0 = L.BatchNormalization(512)
            self.bn1 = L.BatchNormalization(256)
            self.bn2 = L.BatchNormalization(128)
            self.bn3 = L.BatchNormalization(64)


    def __call__(self, z):
        h = self.l0(z)
        h = F.reshape(h, (-1, 512, 3, 3))
        h = F.relu(self.bn0(h))
        h = F.relu(self.bn1(self.deconv0(h)))
        h = F.relu(self.bn2(self.deconv1(h)))
        h = F.relu(self.bn3(self.deconv2(h)))
        x = F.sigmoid(self.deconv3(h))
        return x


class Discriminator(chainer.Chain):
    def __init__(self):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.conv0 = L.Convolution2D(in_channels=1, out_channels=64, ksize=3, stride=3, pad=1)
            self.conv1 = L.Convolution2D(64, 128, 2, 2, 1)
            self.conv2 = L.Convolution2D(128, 256, 2, 2, 1)
            self.conv3 = L.Convolution2D(256, 512, 2, 2, 1)
            self.l0 = L.Linear(in_size=None, out_size=2)
            self.bn0 = L.BatchNormalization(128)
            self.bn1 = L.BatchNormalization(256)
            self.bn2 = L.BatchNormalization(512)


    def __call__(self, x):
        h = F.leaky_relu(self.conv0(x))
        h = F.leaky_relu(self.bn0(self.conv1(h)))
        h = F.leaky_relu(self.bn1(self.conv2(h)))
        h = F.leaky_relu(self.bn2(self.conv3(h)))
        y = self.l0(h)

        return y


In [0]:
class Training:
    def __init__(self, GPU=True):
        self.generator = Generator()
        self.discriminator = Discriminator()
        self.gen_opt = Adam(beta1=0.5)
        self.dis_opt = Adam(beta1=0.5)
        self.gen_opt.setup(self.generator)
        self.dis_opt.setup(self.discriminator)
        self.generator.to_gpu()
        self.discriminator.to_gpu()

    def test(self, z):
        y = self.generator(z).data.get()
        show_img(y)

    def batch(self, z, x):
        fakeimg = self.generator(z)
        y_fake = self.discriminator(fakeimg)
        y_real = self.discriminator(x)
        
        gen_loss = F.softmax_cross_entropy(y_fake, cp.ones(len(z), dtype=cp.int8))
        
        dis_y = F.concat([y_fake, y_real], axis=0)
        dis_t = cp.hstack([cp.zeros(len(z), dtype=cp.int8), cp.ones(len(x), dtype=cp.int8)])
        dis_loss = F.softmax_cross_entropy(dis_y, dis_t)
        

        
        self.generator.cleargrads()
        gen_loss.backward()
        self.gen_opt.update()
        
        self.discriminator.cleargrads()
        dis_loss.backward()
        self.dis_opt.update()
        
        return gen_loss.data, dis_loss.data
        
        

    def training(self, epoch, batchsize):
        test_z = cp.random.uniform(-1, 1, (10, 100, 1, 1)).astype(cp.float32)
        for i in range(epoch):
            index = np.random.permutation(len(trainX))
            for j in range(0, len(trainX), batchsize):
                x = cp.asarray(trainX[index[j:j+batchsize]]).astype(np.float32).reshape(batchsize, 1, 28, 28)
                z = cp.random.uniform(-1, 1, (batchsize, 100, 1, 1)).astype(cp.float32)
                gen_loss, dis_loss = self.batch(z, x)
                if j%10000==0:
                    print(f"epoch:{i:3d} batch:{j:6d} gen_loss:{float(gen_loss):0.4f} dis_loss:{float(dis_loss):0.4f}")
            self.test(test_z)



In [0]:
hoge = Training()
hoge.training(200, 50)